# Single ARIMAX

---

In [ ]:
# DataFrame
import pandas as pd
import numpy as np
import random
from datetime import date

# Visualization
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.simplefilter(action='ignore')

# Save the log
import os
import pickle
import time

# ARIMA
from pmdarima.arima import auto_arima

# Metric 
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
# set the seed
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
# Minus
matplotlib.rcParams['axes.unicode_minus'] = False
# font
plt.rc('font', family='Serif')

In [ ]:
def split_data(product_df):
    
    train_df = product_df[product_df['Date']<'2022-08-01'].reset_index(drop=True)
    test_df = product_df[product_df['Date']>='2022-08-01'].reset_index(drop=True)
    return train_df, test_df

In [ ]:
# Rolling Forecasting
def ARIMA_single(product_df):

    train_df, test_df = split_data(product_df)
    features = product_df.drop(['y', 'Date','Product', '년월'], axis=1).columns.tolist()
    predictions = []
    best_model = auto_arima(train_df['y'], X=train_df[features],
                            start_p=1, start_q=1,
                            max_p=3, max_q=3, 
                            max_d=1, trace=True,
                            suppress_warnings=True)
    
    for test_date in test_df['Date']:
        print(test_date)
        
        test_date_prev_1 = test_date - pd.Timedelta(days=1)
        train_until_test_df = product_df[product_df['Date'] <= test_date_prev_1]
        
        best_model_fit = best_model.fit(train_until_test_df['y'], X=train_until_test_df[features])
        
        # Using exogenous variables of the day before the test date for prediction
        exog_for_pred = train_until_test_df[train_until_test_df['Date'] == test_date_prev_1][features].values.reshape(1, -1)
        prediction = best_model_fit.predict(n_periods=1, X=exog_for_pred).iloc[0]
    
        predictions.append(max(0, prediction))

    # Create a DataFrame to hold the result
    res_df = test_df.copy()
    res_df['Pred'] = predictions
    res_df.set_index('Date', inplace=True)

    # res_df: ['y'','Pred'] index='Date'
    return res_df

In [ ]:
def save_model(product_code, best_model):
    today = date.today()
    folder_path = 'Result/ARIMAX/Model'
    file_name = f'{product_code}_{today.month:02d}{today.day:02d}.pkl'
    save_path = os.path.join(folder_path, file_name)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    # save the model
    with open(save_path, 'wb') as f:
        pickle.dump(best_model, f)
    return best_model

In [ ]:
def load_model(file_name):
    file_path = f'Result/ARIMAX/Model/{file_name}'
    
    with open(file_path, 'rb') as file:
        model_dict= pickle.load(file)
    
    return best_model

In [ ]:
def actual_pred_plot(product_code, res_df, metric_df):
    """
    Plot the actual vs predition and save the figure in the given directory
    """
    save_path = os.path.join("Result", "ARIMAX", product_code)
    save_name = f'{product_code}_all_result'
    
    title = f"Pred Actual Plot - {product_code}"
    actual = res_df['y']
    pred = res_df['Pred']

    # Plot   
    plt.figure(figsize=(16, 8))
    plt.title(title, fontsize=20)
    plt.xlabel("Date", fontsize=14)
    plt.ylabel("Order Demand", fontsize=14)
    plt.plot(actual, label ='Actual', color='r')
    plt.plot(pred, label='Prediction', color='b')
    plt.legend(loc="upper right")
        
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    plt.savefig(os.path.join(save_path, save_name+'.png'))
    plt.show()
    
    metric_df.to_csv(os.path.join(save_path, f'{product_code}_metric.csv'), encoding="utf-8-sig")
    res_df.to_csv(os.path.join(save_path, f'{product_code}_total_result.csv'), encoding="utf-8-sig")
    plt.close('all') # close all figures to free up memory

## Metrics

In [ ]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / (actual+1)))

def nrmse(y_true, y_pred):
    mse =  root_mean_squared_error(y_true, y_pred)
    target_mean = np.mean(y_true)
    nrmse = mse / target_mean
    return nrmse

def nmae(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    target_mean = np.mean(y_true)
    nmae = mae / target_mean
    return nmae

In [ ]:
def calculate_metrics(product_code, res_df):

    actual = res_df['y']
    pred = res_df['Pred']

    MAPE = mape(actual, pred) 
    RMSE = root_mean_squared_error(actual, pred)
    MAE = mean_absolute_error(actual,pred) 
    NRMSE = nrmse(actual,pred) 
    NMAE = nmae(actual,pred) 
    R2 = r2_score(actual, pred)

    metric_df = pd.DataFrame({'MAPE':[round(MAPE, 4)],
                           'RMSE':[round(RMSE, 4)],
                           'MAE':[round(MAE, 4)],
                           'NRMSE':[round(NRMSE, 4)],
                           'NMAE':[round(NMAE, 4)],
                           'R2': [round(R2, 4)]},
                            index= [product_code])

    return metric_df

In [ ]:
def execute_single_ARIMA(product_code):
    start_time = time.time()
    
    product_code = product_code 
    product_df = df[df['Product']== product_code].reset_index(drop=True)

    res_df = ARIMA_single(product_df) 

    metric_df= calculate_metrics(product_code, res_df)
    actual_pred_plot(product_code, res_df, metric_df)
    
    elapsed_time_seconds = time.time() - start_time
    elapsed_time_minutes = elapsed_time_seconds / 60
    print("실행 시간: {:.2f} 분".format(elapsed_time_minutes))
    return metric_df

---

In [ ]:
df = pd.read_csv("../Data/dataset.csv")
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
set_seed(1234)
all_metric = pd.DataFrame()

target_code = ['Office Product', 'Packaging material', 'Pharmaceuticals']
for code in target_code:
    print("==================================")
    print(f"========== { code } ==========")
    print("==================================")
    tmp_metric = execute_single_ARIMA(code)
    all_metric = pd.concat([all_metric, tmp_metric])